In [11]:
# LONGEST VOCAB TOKEN

import pickle

with open("../out/tokenizers/owt_train-merges.txt", "rb") as f:
    merges = pickle.load(f)

with open("../out/tokenizers/owt_train-vocab-32000.txt", "rb") as f:
    vocab = pickle.load(f)

print(merges[:10])
print(list(vocab.items())[1000:1100])

longest_vocab_token: bytes = sorted(list(vocab.values()), key=lambda x: len(x), reverse=True)[0]
print(longest_vocab_token)
print(longest_vocab_token.decode("utf-8"))

[(b' ', b't'), (b' ', b'a'), (b'h', b'e'), (b'i', b'n'), (b'r', b'e'), (b' t', b'he'), (b'o', b'n'), (b'e', b'r'), (b' ', b's'), (b' ', b'w')]
[(1000, b'way'), (1001, b' government'), (1002, b'gg'), (1003, b' Re'), (1004, b'led'), (1005, b'ner'), (1006, b'hip'), (1007, b'atch'), (1008, b'hes'), (1009, b'ames'), (1010, b'any'), (1011, b' ter'), (1012, b' somet'), (1013, b'cent'), (1014, b'velop'), (1015, b'ted'), (1016, b'We'), (1017, b' ref'), (1018, b' trans'), (1019, b'ten'), (1020, b' ins'), (1021, b' run'), (1022, b' ser'), (1023, b' found'), (1024, b' three'), (1025, b'ives'), (1026, b'ave'), (1027, b' mem'), (1028, b' around'), (1029, b' here'), (1030, b'ather'), (1031, b' med'), (1032, b' real'), (1033, b' used'), (1034, b'ouse'), (1035, b' 10'), (1036, b'ved'), (1037, b' exp'), (1038, b'az'), (1039, b' Ar'), (1040, b'cept'), (1041, b'oot'), (1042, b' system'), (1043, b' You'), (1044, b'ci'), (1045, b' really'), (1046, b' mod'), (1047, b'ving'), (1048, b'con'), (1049, b'cer'), (

In [ ]:
data_dir = None # NOTE: set your data dir path

In [13]:
# COMPRESSION RATIO

from cs336_basics.tokenizer import Tokenizer

tokenizer = Tokenizer.from_files("../out/tokenizers/TinyStoriesV2-GPT4-train-vocab-10000.txt", "../out/tokenizers/TinyStoriesV2-GPT4-train-merges.txt", ["<|endoftext|>"])
# tokenizer = Tokenizer.from_files("../out/tokenizers/owt_train-vocab-32000.txt", "../out/tokenizers/owt_train-merges.txt", ["<|endoftext|>"])

data_path = data_dir + "TinyStoriesV2-GPT4-valid.txt"
# data_path = data_dir + "owt_valid.txt"

with open(data_path) as f:
    contents = f.read()

# Sample 10 docs
docs = contents.split("<|endoftext|>")[:10]

ids = []
doc_bytes = []

for doc in docs:
    # print(doc)
    ids.extend(tokenizer.encode(doc))
    doc_bytes.extend(bytes(doc, encoding="UTF-8"))

num_tokens = len(ids)
num_bytes = len(doc_bytes)

print(num_bytes / num_tokens)

4.012658227848101


In [14]:
# FREQUENCY OF TOKEN IN DATASET

target_bytes = b"\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82"
count = 0

data_path = data_dir + "owt_train.txt"

with open(data_path, "rb") as f:
    buffer = b""
    chunk_size = 1024 * 1024
    while True:
        chunk = f.read(chunk_size)
        if not chunk:
            break
        buffer += chunk
        count += buffer.count(target_bytes)
        buffer = buffer[-len(target_bytes) :]

print(f"Occurrences: {count}")

Occurrences: 4679


In [15]:
# TOKENIZER THROUGHPUT IN BYTES/SECOND

import time
from cs336_basics.tokenizer import Tokenizer
import os

tokenizer = Tokenizer.from_files("../out/tokenizers/TinyStoriesV2-GPT4-train-vocab-10000.txt", "../out/tokenizers/TinyStoriesV2-GPT4-train-merges.txt", ["<|endoftext|>"])

data_path = data_dir + "TinyStoriesV2-GPT4-valid.txt"
file_size = os.path.getsize(data_path) # in bytes
ct = 0
with open(data_path) as f:
    token_stream = tokenizer.encode_iterable(f)

    start_time = time.time()
    while next(token_stream, -1) != -1:
        ct += 1
    end_time = time.time()

total_time = end_time - start_time
bytes_per_second = file_size / total_time

print(f"{ct:,} tokens processed")
print(f"{file_size:,} bytes processed")
print(f"{bytes_per_second:,.2f} bytes/s")
print(f"{ct / total_time:,.2f} tok/s")
print(f"{tokenizer.cache_hits / ct:.2%} cache hits")

5,461,204 tokens processed
22,502,601 bytes processed
3,378,131.08 bytes/s
819,845.80 tok/s
98.90% cache hits
